**Generator Module**

In [1]:
def clean_to_pass(para):   #Module to get the read text from the user and convert it into sentence for comparison
    import re
    import string
    import nltk
    stopwords = nltk.corpus.stopwords.words('english')
    para = "".join([i for i in para if i not in string.punctuation])
    para = "".join([i.lower() for i in para if i not in string.punctuation])
    para = para.replace('\n',' ')
    para = para.replace("%"," Percentage")
    return para

In [21]:
def gpt2_generator():
    import torch
    from transformers import GPT2LMHeadModel, GPT2Tokenizer
    # initialize tokenizer and model from pretrained GPT2 model
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

    model = GPT2LMHeadModel.from_pretrained('gpt2')


    list_of_sequences = [
    'Out of office during the 1930s, Churchill took the lead in calling for British rearmament to counter the growing threat of militarism in Nazi Germany. At the outbreak of the Second World War he was re-appointed First Lord of the Admiralty. In May 1940, he became Prime Minister, replacing Neville Chamberlain. Churchill oversaw British involvement in the Allied war effort against the Axis powers, resulting in victory in 1945. After the Conservatives defeat in the 1945 general election, he became Leader of the Opposition',
    'Artificial intelligence was founded as an academic discipline in 1955, and in the years since has experienced several waves of optimism,followed by disappointment and the loss of funding (known as an "AI winter"), followed by new approaches, success and renewed funding. After AlphaGo successfully defeated a professional Go player in 2015, artificial intelligence once again attracted widespread global attention',
    'Technology has many effects. It has helped develop more advanced economies (including todays global economy) and has allowed the rise of a leisure class. Many technological processes produce unwanted by-products known as pollution and deplete natural resources to the detriment of Earths environment. Innovations have always influenced the values of a society and raised new questions in the ethics of technology. Examples include the rise of the notion of efficiency in terms of human productivity, and the challenges of bioethics',
    'Nature has furnished us with numerous significant assets like water, air, food, therapeutic plants, fire, fuels, woods, and trees, and so forth. Every one of these assets helps us for different purposes. As a whole, we realize that our endurance will turn out to be difficult or end without them. As they serve us, they likewise need our adoration and service.',
    'We should love every creation of God, regardless of whether a plant or a creature. We should treat them with affection, and attempt to secure them, however much as it is possible to do. Likewise, cherishing nature incorporates keeping the air and water clean, planting numerous trees, taking care of the variety of animals, treating harmed creatures or helpless humans, and so on. Our acts of love will include a touch of paradise the earth',
    'Everybody makes a few companions at various phases of life. A few companionships last more while some end in only a couple of months. The fundamental explanation for it is the presence of love. Indeed, even we ought to have a sentiment of affection for the needy and poor. They deserve our love the most. We should that is why show our love and affection to everybody irrespective of any differences among each other'
                    ]
    import random
    sequence = list_of_sequences[random.randint(0, 5)]
    
    inputs = tokenizer.encode(sequence, return_tensors='pt')

    # we pass a maximum output length of 200 tokens
    outputs = model.generate(inputs, max_length=200, do_sample=True)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    clean_text = clean_to_pass(text)

    return {
    'to_read':text,
    'to_check':clean_text
    }

In [44]:
text_dict = gpt2_generator()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [45]:
print("Read The Below Sentence after the timer goes off,ignore the complex/unwanted Characters")

Read The Below Sentence after the timer goes off,ignore the complex/unwanted Characters


In [46]:
print("                         ")

In [47]:
print(text_dict['to_read'])

Out of office during the 1930s, Churchill took the lead in calling for British rearmament to counter the growing threat of militarism in Nazi Germany. At the outbreak of the Second World War he was re-appointed First Lord of the Admiralty. In May 1940, he became Prime Minister, replacing Neville Chamberlain. Churchill oversaw British involvement in the Allied war effort against the Axis powers, resulting in victory in 1945. After the Conservatives defeat in the 1945 general election, he became Leader of the Opposition and the first British Prime Minister since Lord Rees in 1861. A strong opponent of the war in the Second World War, he was in office from 1930 to 1939 — the first prime minister, in history, to be removed from office.

In his later years, Churchill supported the British expansionist program of the National Socialist Workers Movement in Germany from 1948 to 1945 for the country's defence against international Communism. The country joined the Allies in the League of Nation

**Recording Phase**

In [48]:
import time
def countdown(t):
    
    while t:
        mins, secs = divmod(t, 60)
        timer = '{:02d}:{:02d}'.format(mins, secs)
        print(timer, end="\r")
        time.sleep(1)
        t -= 1
      
    print('Read it out!')

countdown(10)


Read it out!


In [59]:
def read_out():
    import pyaudio
    import wave

    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    RECORD_SECONDS = 10
    WAVE_OUTPUT_FILENAME = "output.wav"

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* recording")
    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    
    print("* done recording")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [64]:
read_out()

* recording
* done recording


**Convertion Module** 
- Text  ( intelligence score  1/ 0 )
- Spectrogram

1.) Text - Issue with API Module

In [81]:
import speech_recognition as sr
r = sr.Recognizer()

In [82]:
hellow=sr.WavFile('output.wav')
with hellow as source:
    audio = r.record(source)

try:
    s = r.recognize(audio).lower()
    print("Text: "+s)
    intelligence_score = 1
except Exception as e:
    print("Exception: "+str(e))
    intelligence_score = 0
    print("Final Try? Read Again, Ensure a Calm Environment / API Error")
    s = text_dict['to_read'][:250]

Exception: Speech is unintelligible
Final Try? Read Again, Ensure a Calm Environment / API Error


In [83]:
s

'Out of office during the 1930s, Churchill took the lead in calling for British rearmament to counter the growing threat of militarism in Nazi Germany. At the outbreak of the Second World War he was re-appointed First Lord of the Admiralty. In May 194'

2.) Spectrogram

In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
from memory_profiler import memory_usage
from matplotlib import figure
import gc
from pathlib import Path
import librosa
import librosa.display
import pylab

In [109]:
#use the same function , by changing the directory for test set creating
def create_spectrogram_images(filename,name):
    count_ = 0
    plt.interactive(False)
    clip,sample_rate = librosa.load(filename,sr=None)
    fig = plt.figure(figsize=[0.74,0.74])
    
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    
    s = librosa.feature.melspectrogram(y=clip,sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(s,ref=np.max))
    
    filename =  name + '.jpg'
    count_+=1
    plt.savefig(filename,dpi=400,bbox_inches='tight',pad_inches=0)
    plt.close()
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,s
    
def outside_fn():
    
    file = "output.wav"
    print(file)
    filename,name = file,file.split('/')[-1].split('.')[0]
    print(filename)
    name = str("output")
    print(name)
    create_spectrogram_images(filename,name)
    
    

In [110]:
outside_fn()

output.wav
output.wav
output


In [112]:
# Outputs:
# text: s 
# image: output.jpg